In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.regression import LabeledPoint
from sklearn import metrics
from numpy import array
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import LogisticRegressionWithSGD, LogisticRegressionModel
sc = SparkContext(appName="ClassificationOfWineDataset")

In [2]:
#getting data
data = sc.textFile("D://NEU - Big Data and Intelligent Analytics/Assignment 1/winequality-white-classification-parsed format.csv")

In [3]:
def line_to_field(line):
    #space_separated_line = line.replace(';', ' ')
    string_array = line.split(',')
    #float_array = map(float, string_array)
    #float_array[11] = np.where(float_array[10] > 5.0, 1, 0)
    #float_array[11]=(float_array[10] > 5) 
    return LabeledPoint(string_array[0], string_array[1].split(' '))


In [53]:
parsedData = data.map(line_to_field)

In [54]:
#Spliting data into training and test data
(training, test) = parsedData.randomSplit([0.7, 0.3])

In [62]:
# Build the model
model = LogisticRegressionWithSGD.train(training, iterations=200,step=0.1,regParam= 0.01)

In [63]:
# Evaluating the model on training data
labelsAndPreds = training.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_training = labelsAndPreds.filter(lambda (v, p): v == p).count() / float(training.count())
print("Accuracy(training) = " + str(accuracy_training))
x = np.array(labelsAndPreds.collect())

Accuracy(training) = 0.563311220075


In [64]:
# Evaluating the model on test data
labelsAndPreds1 = test.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_test = labelsAndPreds1.filter(lambda (v, p): v == p).count() / float(test.count())
print("Accuracy(test) = " + str(accuracy_test))
y = np.array(labelsAndPreds1.collect())

Accuracy(test) = 0.589797344514


In [65]:
#clearing the threshold for getting scores as prediction.
model.clearThreshold()

In [66]:
#making an array of true lables and its predicted scores for training data
scoreandlabels = np.array(training.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Training Area under curve  = " + str(roc_auc_score(scoreandlabels[:,0],scoreandlabels[:,1]))
print "Training Area under Precision-Recall = " + str(average_precision_score(scoreandlabels[:,0],scoreandlabels[:,1]))

Training Area under curve  = 0.67741902931
Training Area under Precision-Recall = 0.326743663734


In [67]:
#making an array of true lables and its predicted scores for test data
scoreandlabels1 = np.array(test.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Test Area under curve  = " + str(roc_auc_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))
print "Test Area under Precision-Recall = " + str(average_precision_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))

Test Area under curve  = 0.690089434107
Test Area under Precision-Recall = 0.382073277092


In [68]:
#Making confusion matrix for training data
confusion_matrix(x[:,0],x[:,1])

array([[1403, 1342],
       [ 172,  550]])

In [69]:
#Making confusion matrix for test data
confusion_matrix(y[:,0],y[:,1])

array([[580, 513],
       [ 74, 264]])

In [70]:
#checking the weights for the various features
model.weights

DenseVector([0.1864, 0.0043, 0.0065, -0.2343, -0.0024, 0.3048, -0.1414, 0.0362, 0.1356, 0.0255, 0.8417])

In [220]:
sc.stop()